In [1]:
import os,time,sys
import ghtest.param_suggestor as ps
from ghtest import make_test, scan, suggest, write_module
from ghtest.tests_writer import TestArtifact
#ps.VB=1

In [2]:
# set print behavior
from IPython.display import display, Javascript
display(Javascript("""
document.querySelectorAll('.jp-OutputArea-child').forEach(e => e.style.maxHeight = 'none');
"""))

<IPython.core.display.Javascript object>

In [3]:
# input
cassette_dir='tests/cassettes'
test_dir='testdata_tests/'
#src_folder='../temp_folder/src/apigh/'
src_folder='testdata'
#test_folder='../temp_folder/tests/'
safe_switch='GHTEST_ASSUME_SAFE'
unsafe='1'

In [4]:
os.environ[safe_switch]=unsafe

In [5]:
# remove param_db and param history
remove_param_db=True
param_db_path='../../.local/state/ghtest/param_db.json'
param_hist_path='../../.local/state/ghtest/param_history.json'
pps=[param_db_path, param_hist_path]
for pt in pps:
    if os.path.isfile(pt):
        os.remove(pt)
        bn=os.path.basename(pt)
        print(f'{bn} removed')

param_db.json removed
param_history.json removed


In [6]:
# scan 
scs=scan(src_folder)
#sct=scan(test_folder)len(scs)#, len(sct)

In [7]:
# filter non-api and test methods
funcs=scs
#funcs=[f for f in funcs if not f.qualname.startswith('_')]
funcs=[f for f in funcs if not f.qualname.startswith('test_')]
#funcs=scs
#funcs=funcs[0:2]
for func in funcs:
    print(func.qualname )

_remove_auth
_get_env_value
_get_user
_get_token
_get_headers
_paginate
get_commits
list_repos
create_repo
delete_repo
get_repo_data


In [8]:
# check functions for certain values (which should have been removed)
for func in funcs:
    #print(func.qualname)
    [p for p in func.parameters if p.name=='user']
    {k:v for k,v in func.module_globals.items() if 'user' in k}
    [c for c in func.sample_calls if any('user' in k for k in c)]
    [c for c in func.module_call_values if any('user' in k for k in c)]
    {k:v for k,v in func.module_param_values.items() if 'user' in k}
    
    #func.crud_role
    #func.crud_resource
    #[[(p.name, p.default) for p in f.parameters if 'user' in p.name] for f in func.module_functions]
    #[f for f in [[(p.name, p.default) for p in f.parameters if p.default and 'k4144' in p.default] for f in func.module_functions] if f]
    vars(func).keys()
    keys=['module', 'qualname', 'filepath', 'lineno', 'parameters', 'returns', 'docstring', 'module_globals', 'sample_calls', 'module_call_values', 'module_param_values', 'crud_role', 'crud_resource', 'module_functions']
    for k in keys:
        s=eval(f'func.{k}')
        if 'k4144' in str(s).lower():
            pass
            print(k)
    if 'k4144' in str(func).lower():
        pass
        print('k4144')

In [9]:
sps=[]
for func in funcs:
    sp=suggest(func,test_functions=None, literal_only=True)
    print(sp.module, sp.qualname)
    for ps in sp.param_sets:
        print(ps)
    sps.append(sp)

gh_api _remove_auth
{'headers': {'key': 'value'}}
gh_api _get_env_value
{}
{'env_path': 'example.txt_alt'}
{'term': 'GH_USER_TEST'}
gh_api _get_user
{'env_path': 'example.txt_alt'}
{}
{'user_key': 'example'}
gh_api _get_token
{}
{'env_path': 'example.txt_alt'}
{'api_key_value': 'example'}
gh_api _get_headers
{}
{'vb': 0}
{'vb': 1}
gh_api _paginate
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}}
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 'max'}
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 29}
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 30}
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 31}
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'vb': 0}
{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'vb': 1}
{'url': 'https://api.github.com/user/repos'

In [10]:
# change param
# for security, we change 'coron' to coron7
param_update={'name':'coron7'}
for sp in sps:
    updated_param_set=[]
    for op in sp.param_sets:
        for k,v in param_update.items():
            if k in op:
                op=op|param_update
        updated_param_set.append(op)
    sp.param_sets=updated_param_set
    print(sp.qualname)
    print(sp.param_sets)
    print()

_remove_auth
[{'headers': {'key': 'value'}}]

_get_env_value
[{}, {'env_path': 'example.txt_alt'}, {'term': 'GH_USER_TEST'}]

_get_user
[{'env_path': 'example.txt_alt'}, {}, {'user_key': 'example'}]

_get_token
[{}, {'env_path': 'example.txt_alt'}, {'api_key_value': 'example'}]

_get_headers
[{}, {'vb': 0}, {'vb': 1}]

_paginate
[{'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}}, {'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 'max'}, {'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 29}, {'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 30}, {'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'max_items': 31}, {'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'vb': 0}, {'url': 'https://api.github.com/user/repos', 'headers': {'key': 'value'}, 'vb': 1}, {'url': 'https://api.github.com/user/repos', 'headers'

In [11]:
# make tests
remove_cassette=False
gts=[]
cfs=[os.path.join(cassette_dir,f) for f in os.listdir(cassette_dir)]
nr=0
for sp in sps:
    gt=make_test(suggestion=sp, cassette_dir=cassette_dir)
    gts.append(gt)
    cp=gt.cassette_path
    bn=os.path.splitext(cp)[0]
    if remove_cassette:
        for f in cfs:
            if bn in f and os.path.isfile(f):
                os.remove(f)
                nr+=1
print(f'{nr} files deleted from {cassette_dir}')

0 files deleted from tests/cassettes


In [12]:
# remove all previous tests 
remove_tests=True
import shutil
nf,nd=0,0
for file in os.listdir(test_dir):
    path=os.path.join(test_dir, file)
    if os.path.isfile(path):
        os.remove(path)
        nf+=1
    elif os.path.isdir(path):
        shutil.rmtree(path)
        nd+=1
print(f'{nf} files, {nd} folders deleted from {test_dir}')

8 files, 2 folders deleted from testdata_tests/


In [16]:
trs=[]
for gt in gts:
    try:
        tr=gt.test_callable()
        trs.append(tr)
    except Exception as e:
        print(str(e))

In [17]:
#GHTEST_ASSUME_SAFE

In [15]:
# sanitize cassettes
from ghtest import cassette_sanitizer as cs
args=['tests/cassettes',]
r=cs.main(args)
r

updated 53 cassette(s) out of 53


0

In [18]:
artifacts=[]
rrs=[]
for sp,tr in zip(sps, trs):
    artifacts.append(TestArtifact(suggestion=sp, run=tr))
r=write_module(artifacts=artifacts, output_dir=test_dir)
r

TestWriterResult(test_modules=[PosixPath('testdata_tests/test_generated_0.py'), PosixPath('testdata_tests/test_generated_1.py'), PosixPath('testdata_tests/test_generated_2.py'), PosixPath('testdata_tests/test_generated_3.py'), PosixPath('testdata_tests/test_generated_4.py'), PosixPath('testdata_tests/test_generated_5.py'), PosixPath('testdata_tests/test_generated_6.py'), PosixPath('testdata_tests/test_generated_7.py')], scenario_modules=[PosixPath('testdata_tests/scenarios/scenario_repo_0.py')])